<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/Convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2025-05-13 02:35:16--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2025-05-13 02:35:17 (45.3 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a51d633e/releases/pybi

In [ ]:
# @title Import required modules
import os
WD_PATH = "" # If you're using the default Colab runtime, this is just where the runtime is located
#DATA_FILE_PATH = os.path.join(WD_PATH, "aorta_test.vtk")

# Libraries
import copy
import datetime
import importlib
import json
import matplotlib.pyplot as plt
import meshio
import networkx as nx
import numpy as np
import scipy
import vtk

from dolfin import *
from graphnics import *
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
from vtk.util.numpy_support import vtk_to_numpy
from xii import *

/usr/local/lib/python3.11/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing


In [ ]:
# @title define functions

import vtk
import meshio

from dolfin import *
from vtk.util.numpy_support import vtk_to_numpy
from xii import *
from graphnics import *
from scipy.spatial import cKDTree

def save_mesh_vtk_dg(Lambda, file_path, radius_map, uh1d_dg=None):
    """
    Saves a tube mesh as a VTK file with the option to include the 1D solution as a data array.

    Args:
        Lambda (dolfin.Mesh): The mesh to be saved.
        file_path (str): The path where the VTK file will be saved.
        radius_map (dolfin.UserExpression): A function to compute radius values at each point.
        uh1d (dolfin.Function, optional): A function representing 1D pressure data.

    Returns:
        None
    """
    points = Lambda.coordinates()
    cells = {"line": Lambda.cells()}

    radius_values = np.array([radius_map(point) for point in points])
    uh1d_values = np.array([uh1d_dg(point) for point in points])

    if uh1d_dg is not None:
        mesh = meshio.Mesh(points, cells, point_data={"radius": radius_values, "Pressure1D": uh1d_values})
    else:
        mesh = meshio.Mesh(points, cells, point_data={"radius": radius_values})
    mesh.write(file_path)

    reader = vtk.vtkUnstructuredGridReader()
    reader.SetFileName(file_path)
    reader.Update()

    geometryFilter = vtk.vtkGeometryFilter()
    geometryFilter.SetInputData(reader.GetOutput())
    geometryFilter.Update()

    polydata = geometryFilter.GetOutput()

    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(file_path)
    writer.SetInputData(polydata)
    writer.Write()

def export_all_for_paraview(Lambda, Omega, uh1d_dg, uh3d_dg, radius_map, output_dir="vtk_output"):
    os.makedirs(output_dir, exist_ok=True)

    # -------------------------- #
    # --- Export 1D Vessel ---  #
    # -------------------------- #
    import pyvista as pv
    coords_1d = Lambda.coordinates()

    poly = pv.PolyData()
    poly.points = coords_1d

    # Build polyline segments from graph edges
    connectivity = []
    for i, j in ves_Geom.edges():
        connectivity.extend([2, i, j])  # VTK format: [n_points, pt0, pt1]

    poly.lines = np.array(connectivity)

    # Point data: pressure and radius
    pressure_1d = np.array([uh1d_dg(p) for p in coords_1d])
    radii_1d = np.array([radius_map(p) for p in coords_1d])

    poly["Pressure1D"] = pressure_1d
    poly["radius"] = radii_1d

    mesh1d_path = os.path.join(output_dir, "lambda_1d.vtp")
    poly.save(mesh1d_path)
    print(f"[EXPORT] Saved 1D vessel to: {mesh1d_path}")

    # -------------------------- #
    # --- Export 3D Tissue ---  #
    # -------------------------- #
    points_3d = Omega.coordinates()
    # Extract tetrahedral connectivity
    tetra_cells = [cell.entities(0) for cell in cells(Omega)]
    cells_3d = [("tetra", np.array(tetra_cells))]

    # Get pressure values at vertices
    pressure_3d = uh3d_dg.compute_vertex_values(Omega)

    mesh3d = meshio.Mesh(
        points_3d,
        cells_3d,
        point_data={"Pressure3D": pressure_3d}
    )
    mesh3d_path = os.path.join(output_dir, "omega_3d.vtu")
    mesh3d.write(mesh3d_path)

    print(f"[EXPORT] Saved 1D mesh to {mesh1d_path}")
    print(f"[EXPORT] Saved 3D mesh to {mesh3d_path}")

class radius_function(UserExpression):
    """
    A user expression to compute the radius at a given point based on the nearest control point in a graph.

    Args:
        ves_Geom (graphnics.FenicsGraph): The graph containing control points with radius information.
        mf (dolfin.MeshFunction): A mesh function associated with the graph.
        kdtree (scipy.spatial.cKDTree): A k-d tree for efficient nearest neighbor search in the graph.
        **kwargs: Additional keyword arguments to be passed to the UserExpression constructor.
    """
    def __init__(self, ves_Geom, mf, kdtree, **kwargs):
        self.ves_Geom = ves_Geom
        self.mf = mf
        self.kdtree = kdtree
        super().__init__(**kwargs)

    def eval(self, value, x):
        p = (x[0], x[1], x[2])
        _, nearest_control_point_index = self.kdtree.query(p)
        nearest_control_point = list(self.ves_Geom.nodes)[nearest_control_point_index]
        value[0] = self.ves_Geom.nodes[nearest_control_point]['radius']

    def value_shape(self):
        return ()



In [ ]:
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Convergence for exact solution in Masri et al. 2024

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa = 1.0

mesh_vec = [0,1,2,3]

for i in mesh_vec:
  ncell = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius = 0.05

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda)
  #N = FacetNormal(mesh_3d)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(30.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)


  # === Exact Solutions ===
  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)
  u3_exact_expr = Expression('r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) : (kappa/(1+kappa))*(sin(pi*x[2])+2)',
                             degree = 7,
                             radius = radius,
                             kappa = kappa ,
                             r= r)
  u3_exact = interpolate(u3_exact_expr, V3)

  u1_exact_expr = Expression('(sin(pi*x[2])+2)', degree = 7)
  u1_exact = interpolate(u1_exact_expr, V1)

  vtkfile = File('u3_exact.pvd')
  vtkfile << u3_exact
  vtkfile = File('u1_exact.pvd')
  vtkfile << u1_exact

  # === Forcing terms ===
  f3 = Expression('r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*sin(pi*x[2]) :(kappa/(1+kappa))*pow(pi,2)*sin(pi*x[2])',
                  degree = 4,
                  kappa = kappa,
                  radius = radius,
                  r= r)

  f1 = Expression('area*sin(pi*x[2])*pow(pi,2) + (kappa/(1+kappa))*2*pi*radius*(sin(pi*x[2])+2)',
                  degree = 4,
                  radius = radius,
                  kappa = kappa,
                  area = D_area)

  # === Variational Forms ===
  a00 = inner(grad(u3), grad(v3)) * dxOmega + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda
  a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
  a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
  a11 = D_area * inner(grad(u1), grad(v1)) * dxLambda \
      - D_area * inner(avg(grad(u1)), jump(v1, n)) * dS \
      - D_area * inner(avg(grad(v1)), jump(u1, n)) * dS \
      + (alpha/avg(h)) * inner(jump(u1), jump(v1)) * dS \
      + kappa * D_perimeter * inner(u1, v1) * dxLambda
      #- gamma_in * inner(u1, v1) * D_area * dsLambda(1)
      #+ gamma_out * inner(u1, v1) * D_area * dsLambda(2)

  L0 = inner(f3, v3) * dxOmega
  L1 = inner(f1, v1) * dxLambda
      #- gamma_in * inner(u3_exact_expr, v1) * area * dsLambda(1) \
      #+ gamma_out * inner(u3_exact_expr, v1) * area * dsLambda(2)

  a = [[a00, a01], [a10, a11]]
  L = [L0, L1]

  bc3d = DirichletBC(V3, u3_exact, "on_boundary")
  W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

  A, b = map(ii_assemble, (a, L))
  A, b = apply_bc(A, b, W_bcs)
  A, b = map(ii_convert, (A, b))

  solver = PETScKrylovSolver()
  solver.set_operators(A, A)
  ksp = solver.ksp()

  opts = PETSc.Options()
  opts.setValue('ksp_type', 'gmres')
  opts.setValue('ksp_norm_type', 'unpreconditioned')
  opts.setValue('ksp_atol', 1E-14)
  opts.setValue('ksp_rtol', 1E-30)
  opts.setValue('ksp_monitor_true_residual', None)
  opts.setValue('pc_type', 'hypre')
  ksp.setFromOptions()

  x = b.copy()
  solver.solve(A,x,b)

  wh = ii_Function(W)
  wh.vector()[:] = x
  uh3d, uh1d = wh

  avg_uh3d = Average(uh3d, Lambda, cylinder)
  avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

  # === Store Model Predictions ===
  vtkfile = File('3d_Prediction.pvd')
  vtkfile << uh3d

  vtkfile = File('1d_Prediction.pvd')
  vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)





/usr/local/lib/python3.11/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing
Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_cdd21d347a8b939a40c5cd04a75bdb0e0e4b85cb

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00367975 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.360258 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_99e64e64a0b0bf7efa2745c3166b98a12c5ca04c

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00270343 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0340512 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:---------------------------

...

  0 KSP unpreconditioned resid norm 1.900615789213e+01 true resid norm 1.900615789213e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.876170553108e-02 true resid norm 1.876170553108e-02 ||r(i)||/||b|| 9.871382547471e-04
  2 KSP unpreconditioned resid norm 1.536920023056e-04 true resid norm 1.536920023008e-04 ||r(i)||/||b|| 8.086431943431e-06
  3 KSP unpreconditioned resid norm 1.533064040687e-06 true resid norm 1.533064112321e-06 ||r(i)||/||b|| 8.066144251890e-08
  4 KSP unpreconditioned resid norm 8.477462490504e-09 true resid norm 8.477462617894e-09 ||r(i)||/||b|| 4.460376824189e-10
  5 KSP unpreconditioned resid norm 2.204630837755e-11 true resid norm 2.200210128222e-11 ||r(i)||/||b|| 1.157630143193e-12
  6 KSP unpreconditioned resid norm 1.286682938427e-13 true resid norm 1.530332998677e-13 ||r(i)||/||b|| 8.051774626744e-15
  7 KSP unpreconditioned resid norm 5.764082036162e-15 true resid norm 2.160443176274e-13 ||r(i)||/||b|| 1.136706949682e-14


Averaging over 16 cells: 100%|██████████| 16/16 [00:00<00:00, 411.90it/s]


  0 KSP unpreconditioned resid norm 3.766776560745e+01 true resid norm 3.766776560745e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.090434817840e-02 true resid norm 2.090434817841e-02 ||r(i)||/||b|| 5.549665036218e-04
  2 KSP unpreconditioned resid norm 5.548349946468e-04 true resid norm 5.548349948340e-04 ||r(i)||/||b|| 1.472970286096e-05
  3 KSP unpreconditioned resid norm 1.014837242921e-05 true resid norm 1.014837250684e-05 ||r(i)||/||b|| 2.694179583839e-07
  4 KSP unpreconditioned resid norm 1.196745387122e-07 true resid norm 1.196745397906e-07 ||r(i)||/||b|| 3.177107477991e-09
  5 KSP unpreconditioned resid norm 1.638138921323e-09 true resid norm 1.638175753607e-09 ||r(i)||/||b|| 4.349012284612e-11
  6 KSP unpreconditioned resid norm 1.970430017846e-11 true resid norm 1.973966758849e-11 ||r(i)||/||b|| 5.240466820942e-13
  7 KSP unpreconditioned resid norm 1.836438975039e-13 true resid norm 9.274198909424e-13 ||r(i)||/||b|| 2.462104868676e-14
  8 KSP 

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 448.95it/s]


  0 KSP unpreconditioned resid norm 7.517610382336e+01 true resid norm 7.517610382336e+01 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.099341377946e-02 true resid norm 2.099341377919e-02 ||r(i)||/||b|| 2.792564752826e-04
  2 KSP unpreconditioned resid norm 5.117431894345e-04 true resid norm 5.117431891596e-04 ||r(i)||/||b|| 6.807258731605e-06
  3 KSP unpreconditioned resid norm 1.266395878770e-05 true resid norm 1.266395864763e-05 ||r(i)||/||b|| 1.684572357912e-07
  4 KSP unpreconditioned resid norm 2.021205448808e-07 true resid norm 2.021207477120e-07 ||r(i)||/||b|| 2.688630261911e-09
  5 KSP unpreconditioned resid norm 2.824098803778e-09 true resid norm 2.824105438032e-09 ||r(i)||/||b|| 3.756653104379e-11
  6 KSP unpreconditioned resid norm 3.828452630664e-11 true resid norm 3.835466442426e-11 ||r(i)||/||b|| 5.101975557869e-13
  7 KSP unpreconditioned resid norm 5.288353662949e-13 true resid norm 2.874565380892e-12 ||r(i)||/||b|| 3.823775421571e-14
  8 KSP 

In [ ]:
# @title sanity checks

values = uh1d.vector().get_local()
print(f"1D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = u1_exact.vector().get_local()
print(f"Exact 1D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = uh3d.vector().get_local()
print(f"3D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")
values = u3_exact.vector().get_local()
print(f"Exact 3D pressure range: min = {np.min(values):.3e}, max = {np.max(values):.3e}")

print("Min/max 3D pressure:", uh3d.vector().min(), uh3d.vector().max())
print("Min/max 1D pressure:", uh1d.vector().min(), uh1d.vector().max())

import numpy as np
import matplotlib.pyplot as plt

# Sample points along z-axis at (x=0.5, y=0.5)
num_samples = 100
z_vals = np.linspace(0, 1, num_samples)
x_center = 0.5
y_center = 0.5

u1_exact_values = []
uh1d_values = []

for z in z_vals:
    point = np.array([x_center, y_center, z])
    u1_exact_values.append(u1_exact_expr(point))
    uh1d_values.append(uh1d(point))

# Convert to numpy arrays
u1_exact_values = np.array(u1_exact_values)
uh1d_values = np.array(uh1d_values)

# Plot
plt.figure(figsize=(8,6))
plt.plot(z_vals, u1_exact_values, label='Exact Solution', linestyle='--')
plt.plot(z_vals, uh1d_values, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('z')
plt.ylabel('Pressure at (x=0.5, y=0.5)')
plt.title('Centerline Pressure Comparison (Exact vs Numerical 1D)')
plt.legend()
plt.grid(True)
plt.show()

# Sample points along z-axis at (x=0.5, y=0.5)
num_samples = 100
z_vals = np.linspace(0, 1, num_samples)
x_center = 0.5
y_center = 0.5

u3_exact_values = []
uh3d_values = []

for z in z_vals:
    point = np.array([x_center, y_center, z])
    u3_exact_values.append(u3_exact(point))
    uh3d_values.append(uh3d(point))

# Convert to numpy arrays
u3_exact_values = np.array(u3_exact_values)
uh3d_values = np.array(uh3d_values)

# Plot
plt.figure(figsize=(8,6))
plt.plot(z_vals, u3_exact_values, label='Exact Solution', linestyle='--')
plt.plot(z_vals, uh3d_values, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('z')
plt.ylabel('Pressure at (x=0.5, y=0.5)')
plt.title('Centerline Pressure Comparison (Exact vs Numerical 3D)')
plt.legend()
plt.grid(True)
plt.show()


# Sample points radially outward from center at z=0.5
num_samples_r = 100
r_vals = np.linspace(0, 0.5, num_samples_r)
z_fixed = 0.5

u3_exact_radial = []
uh3d_radial = []

for r in r_vals:
    x = 0.5 + r
    y = 0.5
    point = np.array([x, y, z_fixed])
    u3_exact_radial.append(u3_exact(point))
    uh3d_radial.append(uh3d(point))

# Convert to numpy arrays
u3_exact_radial = np.array(u3_exact_radial)
uh3d_radial = np.array(uh3d_radial)

# Plot
plt.figure(figsize=(8,6))
plt.plot(r_vals, u3_exact_radial, label='Exact Solution', linestyle='--')
plt.plot(r_vals, uh3d_radial, label='Numerical Solution', marker='o', markersize=3, linestyle='-')
plt.xlabel('Radial distance r from center (z=0.5)')
plt.ylabel('Pressure')
plt.title('Radial Pressure Comparison (Exact vs Numerical at z=0.5)')
plt.legend()
plt.grid(True)
plt.show()


1D pressure range: min = 9.959e-01, max = 2.992e+00
Exact 1D pressure range: min = 1.000e+00, max = 3.000e+00
3D pressure range: min = 4.338e-01, max = 1.500e+00
Exact 3D pressure range: min = 4.338e-01, max = 1.500e+00
Min/max 3D pressure: 0.43377103291814956 1.5
Min/max 1D pressure: 0.9959318017399034 2.991558034571256


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     https://fenicsproject.discourse.group/
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to evaluate function at point.
*** Reason:  The point is not inside the domain. Consider calling "Function::set_allow_extrapolation(true)" on this Function to allow extrapolation.
*** Where:   This error was encountered inside Function.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.2.0.dev0
*** Git changeset:  a8dcf753d3ab9ae19d7bedf8c634a4fe545032d2
*** -------------------------------------------------------------------------


In [ ]:
# @title export

# Save numerical 3D solution
uh3d.rename("uh3", "numerical 3D")
File("output/uh3.pvd") << uh3d

# Save exact 3D solution
u3_exact_func = interpolate(u3_exact, uh3d.function_space())
u3_exact_func.rename("u3_exact", "exact 3D")
File("output/u3_exact.pvd") << u3_exact_func

# Save numerical 1D solution
uh1d.rename("uh1", "numerical 1D")
File("output/uh1.pvd") << uh1d

# Save exact 1D solution
u1_exact_func = interpolate(u1_exact, uh1d.function_space())
u1_exact_func.rename("u1_exact", "exact 1D")
File("output/u1_exact.pvd") << u1_exact_func